In [ ]:

#Advanced Time Series Forecasting with LSTM and Attention

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from typing import Tuple, List
import matplotlib.pyplot as plt
def generate_multivariate_series(
    n_samples: int = 3000,
    noise_level: float = 0.2
) -> pd.DataFrame:
    t = np.arange(n_samples)

    f1 = np.sin(0.02 * t)
    f2 = np.cos(0.015 * t)
    f3 = np.sin(0.01 * t) + 0.5 * f1
    f4 = 0.3 * f2 + np.sin(0.05 * t)
    f5 = np.sin(0.03 * t) + np.cos(0.02 * t)
    f6 = f1 * f2

    noise = noise_level * np.random.randn(n_samples, 6)

    data = np.vstack([f1, f2, f3, f4, f5, f6]).T + noise

    columns = [f"feature_{i}" for i in range(6)]
    return pd.DataFrame(data, columns=columns)
def create_sequences(
    data: np.ndarray,
    seq_length: int
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Convert time series into supervised learning sequences.
    """
    X, y = [], []

    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length, 0])  # Predict feature_0

    return np.array(X), np.array(y)
def build_baseline_lstm(
    seq_length: int,
    n_features: int,
    hidden_units: int = 64
) -> Model:
    """
    Build standard LSTM forecasting model.
    """
    inputs = layers.Input(shape=(seq_length, n_features))
    x = layers.LSTM(hidden_units)(inputs)
    outputs = layers.Dense(1)(x)

    model = Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="mse"
    )

    return model
class BahdanauAttention(layers.Layer):
    """
    Implements Bahdanau additive attention.
    """

    def _init_(self, units: int):
        super()._init_()
        self.W1 = layers.Dense(units)
        self.W2 = layers.Dense(units)
        self.V = layers.Dense(1)

    def call(self, hidden_states):
        score = tf.nn.tanh(self.W1(hidden_states))
        attention_weights = tf.nn.softmax(
            self.V(score), axis=1
        )
        context_vector = attention_weights * hidden_states
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights
def build_attention_lstm(
    seq_length: int,
    n_features: int,
    hidden_units: int = 64
) -> Model:
    """
    Build LSTM with Bahdanau Attention.
    """
    inputs = layers.Input(shape=(seq_length, n_features))
    lstm_out = layers.LSTM(
        hidden_units,
        return_sequences=True
    )(inputs)

    attention_layer = BahdanauAttention(hidden_units)
    context_vector, attention_weights = attention_layer(lstm_out)

    outputs = layers.Dense(1)(context_vector)

    model = Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="mse"
    )

    model.attention_layer = attention_layer

    return model
def rolling_origin_splits(
    X: np.ndarray,
    y: np.ndarray,
    n_splits: int = 3
):
    """
    Generator for rolling-origin evaluation.
    """
    fold_size = len(X) // (n_splits + 1)

    for i in range(n_splits):
        train_end = fold_size * (i + 1)
        val_end = fold_size * (i + 2)

        yield (
            X[:train_end],
            y[:train_end],
            X[train_end:val_end],
            y[train_end:val_end]
        )
def train_and_evaluate(
    model_builder,
    X: np.ndarray,
    y: np.ndarray,
    seq_length: int,
    n_features: int
) -> Tuple[float, float]:
    """
    Train model using rolling validation and compute average RMSE and MAE.
    """
    rmses, maes = [], []

    for X_train, y_train, X_val, y_val in rolling_origin_splits(X, y):

        model = model_builder(seq_length, n_features)

        model.fit(
            X_train,
            y_train,
            epochs=20,
            batch_size=32,
            verbose=0
        )

        preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        mae = mean_absolute_error(y_val, preds)

        rmses.append(rmse)
        maes.append(mae)

    return np.mean(rmses), np.mean(maes)
if _name_ == "_main_":

    df = generate_multivariate_series()
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)

    SEQ_LENGTH = 50

    X, y = create_sequences(scaled, SEQ_LENGTH)

    baseline_rmse, baseline_mae = train_and_evaluate(
        build_baseline_lstm,
        X, y,
        SEQ_LENGTH,
        X.shape[2]
    )

    attention_rmse, attention_mae = train_and_evaluate(
        build_attention_lstm,
        X, y,
        SEQ_LENGTH,
        X.shape[2]
    )

    print("Baseline LSTM")
    print("RMSE:", baseline_rmse)
    print("MAE :", baseline_mae)

    print("\nAttention LSTM")
    print("RMSE:", attention_rmse)
    print("MAE :", attention_mae)